# Create a simple ISA descriptor

This example creates minimal metadata for a single study ISA descriptor with no assay declared. 

It shows how to serialize (write) the ISA Model content to ISA-Tab and ISA-JSON formats.

In [176]:
from isatools.model import *
from isatools.create.model import *
import datetime

## Study metadata

In [177]:
investigation = Investigation()
investigation.comments.append(Comment(name="investigation_comment_1", value="inv_comment_value"))
investigation.comments.append(Comment(name="investigation_comment_1", value="inv_comment_value"))

#testing ISA.Comments on Ontology Source Reference: OK
obi = OntologySource(name="obi", file="https://example.org/onto.owl", version="1.0",description="some dummy description")
obi.comments.append(Comment(name="ontosource-com",value="ontosource-com-value"))

otheronto = OntologySource(name="otheronto", file="https://example.org/otheronto.owl", version="3.0",description="some other dummy description")
otheronto.comments.append(Comment(name="otherontosource-com",value="otherontosource-com-value"))
otheronto.comments.append(Comment(name="ontosource-com",value="ontosource-com-value-2"))

investigation.ontology_source_references.append(obi)
investigation.ontology_source_references.append(otheronto)

study = Study(filename="s_study.txt")
study.identifier = "S1"
study.title = "My Simple ISA Study"
study.description = "We could alternatively use the class constructor's parameters to set some default " \
          "values at the time of creation, however we want to demonstrate how to use the " \
          "object's instance variables to set values."


#testing ISA.Comments on ISA.StudyDesignType via ISA.OntologyAnnotation object: OK
oe_design = OntologyAnnotation(term="some design")
oe_design.comments.append(Comment(name="onto_design_com", value="onto_design_value"))
study.design_descriptors.append(oe_design)

#testing ISA.Comments on ISA.Study Object (commentable object): OK
study.comments.append(Comment(name="study", value="study_comment_value"))
study.comments.append(Comment(name="study", value="study_comment_value"))

study.submission_date = str(datetime.datetime.today())
study.public_release_date = str(datetime.datetime.today())

#testing ISA.Comments on ISA.Publication Object (commentable object): OK
pub = Publication(pubmed_id="111111")
pub.comments.append(Comment(name="study_pub_comment",value="study_pub_value"))
study.publications.append(pub)
investigation.publications.append(pub) #testing behavior on both ISA.Investigation and ISA.Study objects

#testing ISA.Comments on ISA.Person Object (commmentable object): OK
buddy = Person(first_name="bob",email="bob.hill@alvaroo.lv")
buddy.comments.append(Comment(name="person_com", value="person_com_value"))
study.contacts.append(buddy)
investigation.contacts.append(buddy) #testing behavior on both ISA.Investigation and ISA.Study objects



study.sources = [Source(name="source1")]
study.samples = [Sample(name="sample1")]

#testing ISA.Comments on ISA.Factor Object (commentable object): OK
# ****WARNING: ISA.Comments can also be passed via the OntologyAnnotation Object on Factor_Type ****: NOT OK
ft_onto=OntologyAnnotation(term="chemical")
ft_com=Comment(name="factor_type_com_via_onto", value="factor_type_com_via_onto_value")
ft_onto.comments.append(ft_com)

study.factors = [StudyFactor(
                             name="Chemical Compound",
                             factor_type=ft_onto,
                             comments=[Comment(name="factor_com",value="some string about factors")])
                ]

#testing ISA.Comments on ISA.Protocol Object (commentable object)
# ****WARNING: ISA.Comments can also be passed via the OntologyAnnotation Object on Protocol.protocol_type ****
# but these aren't currently serialized: NOT OK
study.protocols = [Protocol(name="sample collection"),
                   Protocol(name="extraction",
                            uri="",
                            protocol_type=OntologyAnnotation(term="extraction"),
                            parameters=[ProtocolParameter(parameter_name=OntologyAnnotation(term="genome assembly")),
                                        ProtocolParameter(parameter_name=OntologyAnnotation(term="cut-off value"))]),
                    Protocol(name="data visualization with Intermine",
                             uri="https://intermine.org/10.5464/network.svg",
                             protocol_type=OntologyAnnotation(term="data visualization")),
                    Protocol(name="data acquisition",protocol_type=OntologyAnnotation(term="data acquisition"))
                  ]

#testing ISA.Comments on ISA.Protocol Object (commentable object)
study.protocols[0].comments.append(Comment(name="protocol_com",value="protocol_com_value"))

#testing ISA.Comments on ISA.Protocol.Parameters Object (not a commentable object): OK
# *** IMPORTANT: this call should not be done!
study.protocols[1].parameters[0].comments.append((Comment(name="parameter_com",value="parameter_com_value")))


study.process_sequence = [Process(executes_protocol=study.protocols[-1], inputs=[study.sources[-1]], outputs=[study.samples[-1]])]

char1 = Characteristic(category=OntologyAnnotation(term="organism part"), value=OntologyAnnotation(term="liver"))
char2 = Characteristic(category=OntologyAnnotation(term="amount"), value=10, unit=OntologyAnnotation(term="g"))

study.characteristic_categories.append(char1.category)
study.characteristic_categories.append(char2.category)

study.samples[0].characteristics.append(char1)
study.samples[0].characteristics.append(char2)

# testing ISA.Comments on ISA.ProtocolProcess_Sequence (commentable object): OK
com_process_seq=Comment(name="process_seq_com", value="process_seq_com_value")

extraction_process = Process(executes_protocol=study.protocols[1])
extraction_process.inputs.append(study.samples[-1])
extraction_process.comments.append(com_process_seq)

material = Material(name="extract-1")
material.type = "Extract Name"

# testing ISA.Comments on ISA.Material Object (commentable object): OK
com_mat = Comment(name="amount", value="not much")
material.comments.append(com_mat)

extraction_process.outputs.append(material)


# testing ISA.Comments on ISA.Assay.measurement_type via ISA.OntologyAnnotation object: OK
mt=OntologyAnnotation(term="metabolite profiling")
mt.comments.append(Comment(name="onto-com-mt", value="onto-com-mt-value"))

metprof_assay = Assay(measurement_type=mt,
                      technology_type=OntologyAnnotation(term="mass spectrometry"),filename="a_mp_by_ms.txt")

metprof_assay.samples.append(study.samples[-1])


# testing ISA.Comments on ISA.DataFile Object (commentable object): OK
comdata=Comment(name="Checksum",value="werwtqwteqw23qrwert132")
# metprof_assay.data_files.append(DataFile(filename="sequenced-data-1", label="Raw Data File"))

datafile=DataFile(filename="file-1",label="Spectral Raw Data File")
datafile.comments.append(comdata)
metprof_assay.data_files.append(datafile)

metprof_assay.other_material.append(material)
metprof_assay.process_sequence.append(extraction_process)

da_process = Process(executes_protocol=study.protocols[3],inputs=[material], outputs=[datafile])
da_process.name = "assay-name-test-1"

# testing ISA.Comments on ISA.ProcessSequence Object (commentable object): OK
da_process.comments.append(com_process_seq)
metprof_assay.process_sequence.append(da_process)


# testing ISA.Comments on ISA.Assay Object (commentable object): OK
com_assay = Comment(name="comments on assay", value="good assay?")
metprof_assay.comments.append(com_assay)

plink(extraction_process, da_process)

# testing ISA.Comments on ISA.Study (commentable object): OK
com0 = Comment(name="license", value="CC-BY-SA")
study.comments.append(com0)

# testing ISA.Comments on ISA.Protocol (commentable object): OK
com00 = Comment(name="protocol comment", value="inactive")
study.protocols[0].comments.append(com00)


com000 = Comment(name="parameter comment", value="parameter comment value")

# testing ISA.Comments on ISA.ProtocolParameter (which is NOT a commentable object)
# (SHOULD NOT BE SERIALIZED and AS PREVIOUSLY, THIS CALL IS WRONG)
study.protocols[1].parameters[0].comments.append(com000)



# testing ISA.Comments on ISA.ProtocolProcessSequence (commentable object): OK
study.process_sequence[0].comments.append(com_process_seq)


# testing ISA.Comments on ISA.Source (commentable object): OK
com1 =Comment(name="provenance", value="alpha")
com2 =Comment(name="consent", value="no reuse")
study.sources[0].comments=[com1]
study.sources[0].comments.append(com2)

# testing ISA.Comments on ISA.Sample (commentable object): OK
com3 =Comment(name="regulatory", value="beta")
com4 =Comment(name="abbreviation", value="testing")
study.samples[0].comments.append(com3)
study.samples[0].comments.append(com4)

study.assays.append(metprof_assay)


investigation.studies = [study]

In [178]:
investigation.studies[0].samples


[isatools.model.Sample(name='sample1', characteristics=[isatools.model.Characteristic(category=isatools.model.OntologyAnnotation(term='organism part', term_source=None, term_accession='', comments=[]), value=isatools.model.OntologyAnnotation(term='liver', term_source=None, term_accession='', comments=[]), unit=None, comments=[]), isatools.model.Characteristic(category=isatools.model.OntologyAnnotation(term='amount', term_source=None, term_accession='', comments=[]), value=10, unit=isatools.model.OntologyAnnotation(term='g', term_source=None, term_accession='', comments=[]), comments=[])], factor_values=[], derives_from=[], comments=[isatools.model.Comment(name='regulatory', value='beta'), isatools.model.Comment(name='abbreviation', value='testing')])]

In [179]:
for s in study.sources:
    print(s)
    for c in s.comments:
        print(c)

Source(
    name=source1
    characteristics=0 Characteristic objects
    comments=2 Comment objects
)
Comment(
    name=provenance
    value=alpha
)
Comment(
    name=consent
    value=no reuse
)


In [180]:
study.protocols[1].parameters[0].parameter_name.term

'genome assembly'

In [181]:
# Testing ISA.Comment on ISA.Parameter via Parameter_name~Ontology Annotation: OK in JSON, NOT OK in TAB serialization

study.protocols[1].parameters[0].parameter_name.comments.append(Comment(name="a",value="b"))
study.protocols[1].parameters[0].parameter_name.comments[0].name

'a'

In [182]:
# Let's see the object :
investigation

isatools.model.Investigation(identifier='', filename='', title='', submission_date='', public_release_date='', ontology_source_references=[isatools.model.OntologySource(name='obi', file='https://example.org/onto.owl', version='1.0', description='some dummy description', comments=[isatools.model.Comment(name='ontosource-com', value='ontosource-com-value')]), isatools.model.OntologySource(name='otheronto', file='https://example.org/otheronto.owl', version='3.0', description='some other dummy description', comments=[isatools.model.Comment(name='otherontosource-com', value='otherontosource-com-value'), isatools.model.Comment(name='ontosource-com', value='ontosource-com-value-2')])], publications=[isatools.model.Publication(pubmed_id='111111', doi='', author_list='', title='', status=None, comments=[isatools.model.Comment(name='study_pub_comment', value='study_pub_value')])], contacts=[isatools.model.Person(last_name='', first_name='bob', mid_initials='', email='bob.hill@alvaroo.lv', phone=

## Writing to ISA-Tab

In [183]:
from isatools.isatab import dumps
print(dumps(investigation))

/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/tmpwlck3x0r/i_investigation.txt
ONTOLOGY SOURCE REFERENCE
Term Source Name	obi	otheronto
Term Source File	https://example.org/onto.owl	https://example.org/otheronto.owl
Term Source Version	1.0	3.0
Term Source Description	some dummy description	some other dummy description
Comment[ontosource-com]	ontosource-com-value	ontosource-com-value-2
Comment[otherontosource-com]		otherontosource-com-value
INVESTIGATION
Investigation Identifier	
Investigation Title	
Investigation Description	
Investigation Submission Date	
Investigation Public Release Date	
Comment[investigation_comment_1]	inv_comment_value
Comment[investigation_comment_1]	inv_comment_value
INVESTIGATION PUBLICATIONS
Investigation PubMed ID	111111
Investigation Publication DOI	
Investigation Publication Author List	
Investigation Publication Title	
Investigation Publication Status	
Investigation Publication Status Term Accession Number	
Investigation Publication Status Term Source RE

## Writing to ISA-JSON

In [184]:
import json
from isatools.isajson import ISAJSONEncoder
print(json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "comments": [
        {
            "name": "investigation_comment_1",
            "value": "inv_comment_value"
        },
        {
            "name": "investigation_comment_1",
            "value": "inv_comment_value"
        }
    ],
    "description": "",
    "identifier": "",
    "ontologySourceReferences": [
        {
            "description": "some dummy description",
            "file": "https://example.org/onto.owl",
            "name": "obi",
            "version": "1.0"
        },
        {
            "description": "some other dummy description",
            "file": "https://example.org/otheronto.owl",
            "name": "otheronto",
            "version": "3.0"
        }
    ],
    "people": [
        {
            "address": "",
            "affiliation": "",
            "comments": [
                {
                    "name": "person_com",
                    "value": "person_com_value"
                }
            ],
            "email": "bob.hill@alvaroo.